In [ ]:
!pip install ucimlrepo numpy pandas scikit-learn tensorflow

In [58]:
# ------- Import Dataset -------------------

from ucimlrepo import fetch_ucirepo

# fetch dataset
air_quality = fetch_ucirepo(id=360)

# data (as pandas dataframes)
X = air_quality.data.features
y = air_quality.data.targets # unused?

In [59]:
# Combine Date and Time into a single DateTime column -> only 14 features
import pandas as pd
import numpy as np

X['DateTime'] = pd.to_datetime(X['Date'] + ' ' + X['Time']) # Format (JJJJ-MM-DD HH:MM:SS, e.g.: 2004-03-10 18:00:00)
X = X.set_index('DateTime')
X.drop(['Date', 'Time'], axis=1, inplace=True)  # Remove the original Date and Time columns
X.replace(-200, np.nan, inplace=True)
X.replace(-200.0, np.nan, inplace=True)

In [60]:
pd.set_option('display.width', 1000)
print(X.head())

                     CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)     T    RH      AH
DateTime                                                                                                                                                  
2004-03-10 18:00:00     2.6       1360.0     150.0      11.9         1046.0    166.0        1056.0    113.0        1692.0       1268.0  13.6  48.9  0.7578
2004-03-10 19:00:00     2.0       1292.0     112.0       9.4          955.0    103.0        1174.0     92.0        1559.0        972.0  13.3  47.7  0.7255
2004-03-10 20:00:00     2.2       1402.0      88.0       9.0          939.0    131.0        1140.0    114.0        1555.0       1074.0  11.9  54.0  0.7502
2004-03-10 21:00:00     2.2       1376.0      80.0       9.2          948.0    172.0        1092.0    122.0        1584.0       1203.0  11.0  60.0  0.7867
2004-03-10 22:00:00     1.6       1272.0      51.0       6.5          

In [61]:
# Select only one feature
df = X[['CO(GT)']]
df = df.interpolate()

In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame
# Normalize the data
scaler = MinMaxScaler()
# df['CO(GT)'] = scaler.fit_transform(df[['CO(GT)']])
data = scaler.fit_transform(df)
# Convert DataFrame to a NumPy array
# data = df['CO(GT)'].values
# Function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Define sequence length
seq_length = 5  # Example: 5 time steps

# Prepare the sequences
X_sec, y_sec = create_sequences(data, seq_length)

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_sec, y_sec, epochs=50, batch_size=32)

Epoch 1/50
293/293 [==============================] - 2s 3ms/step - loss: 0.0088
Epoch 2/50
293/293 [==============================] - 1s 3ms/step - loss: 0.0044
Epoch 3/50
293/293 [==============================] - 1s 3ms/step - loss: 0.0035
Epoch 4/50
293/293 [==============================] - 1s 4ms/step - loss: 0.0033
Epoch 5/50
293/293 [==============================] - 1s 3ms/step - loss: 0.0033
Epoch 6/50
293/293 [==============================] - 1s 3ms/step - loss: 0.0032
Epoch 7/50
293/293 [==============================] - 1s 3ms/step - loss: 0.0032
Epoch 8/50
293/293 [==============================] - 1s 4ms/step - loss: 0.0032
Epoch 9/50
293/293 [==============================] - 1s 4ms/step - loss: 0.0032
Epoch 10/50
293/293 [==============================] - 1s 4ms/step - loss: 0.0031
Epoch 11/50
293/293 [==============================] - 1s 3ms/step - loss: 0.0031
Epoch 12/50
293/293 [==============================] - 1s 4ms/step - loss: 0.0031
Epoch 13/50
293/293 [====

In [69]:
from datetime import datetime, timedelta

# Function to predict a future value
def predict_future_value(start_date, end_date, model, last_sequence, scaler):
    current_date = start_date
    while current_date <= end_date:
        # Reshape the sequence for the model
        current_sequence = last_sequence.reshape((1, seq_length, 1))

        # Get the next prediction
        next_prediction = model.predict(current_sequence)[0]

        # Update the sequence to include the new prediction
        last_sequence = np.roll(last_sequence, -1)
        last_sequence[-1] = next_prediction

        # Update the current date
        current_date += timedelta(hours=1)  # adjust the timedelta according to your data's frequency
    
    # Inverse transform the last prediction to get it back to the original scale
    final_prediction = scaler.inverse_transform([next_prediction])[0][0]
    return final_prediction

# Predict for a specific future date
start_date = pd.to_datetime(df.index[-1])  # Last date in your dataset 2005-04-02 14:00:00
end_date = pd.to_datetime("2005-04-12 12:00:00")
last_sequence = data[-seq_length:]  # Last known sequence from your dataset

predicted_value = predict_future_value(start_date, end_date, model, last_sequence, scaler)
print(f"Predicted value at {end_date}: {predicted_value}")

1/1 [==============================] - 0s 35ms/step
Predicted value at 2005-04-13 12:00:00: 3.470724332332611
